In [14]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import os
import string
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import math
import pickle
from PIL import Image, ImageEnhance, ImageOps, UnidentifiedImageError
import cv2
import numpy as np
import requests
from io import BytesIO
import ast
import torch
from torchvision import models, transforms
import asyncio
import aiohttp


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
csv_path = '/content/A2_Data.csv'

df = pd.read_csv(csv_path)
os.makedirs('reviews')
os.makedirs('processed_images')


# **Q1**

In [ ]:
import pandas as pd
import numpy as np
import requests
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image
from io import BytesIO

# Initialize the ResNet50 model for feature extraction
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

def download_image(img_url):
    """
    Download an image from a given URL.
    """
    try:
        response = requests.get(img_url)
        img = Image.open(BytesIO(response.content))
        return img
    except Exception as e:
        print(f"Error downloading {img_url}: {e}")
        return None

def preprocess_image(img):
    """
    Preprocess an image for model input.
    """
    img = img.resize((224, 224))  # Resize the image to match model input dimensions
    img_array = img_to_array(img)  # Convert the image to a numpy array
    expanded_img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return preprocess_input(expanded_img_array)  # Preprocess the image

def extract_features(img_array):
    """
    Extract features from an image array using ResNet50.
    """
    features = model.predict(img_array).flatten()  # Extract features
    normalized_features = features / np.linalg.norm(features)  # Normalize the features
    return normalized_features

# Load the dataset
df = pd.read_csv('/content/A2_Data.csv')

# Iterate over the image URLs and extract features
features_list = []
for index, row in df.iterrows():
    img_urls = eval(row['Image'])  # Convert the string representation of list into an actual list
    if img_urls:  # Check if the list is not empty
        first_img_url = img_urls[0]  # Use the first image URL for feature extraction
        img = download_image(first_img_url)
        if img is not None:
            img_array = preprocess_image(img)
            features = extract_features(img_array)
            features_list.append(features)
        else:
            features_list.append(None)  # Append None if the image couldn't be processed
    else:
        features_list.append(None)  # Append None if there are no image URLs

# Add the extracted features as a new column in the DataFrame
df['Image_Features'] = features_list

# You might want to save the DataFrame with features to a new CSV or a pickle file for future use
df.to_pickle('/content/df_with_image_features.pkl')  # Save to pickle for preserving data type


In [53]:
model = models.resnet50(pretrained=True)
my_model = torch.nn.Sequential(*(list(model.children())[:-1]))
# my_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
Matrix = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def feature_extraction(image_path, neural_net):
    image = Image.open(image_path).convert('RGB')
    preprocessed_image = Matrix(image)
    img_bat = torch.unsqueeze(preprocessed_image, 0)

    with torch.no_grad():
        features = neural_net(img_bat)
    return features.cpu().numpy().flatten()

extracted_features = {}

for image_file in os.listdir('processed_images'):
    if image_file.endswith('.jpg'):
        image_path = os.path.join('processed_images', image_file)
        features = feature_extraction(image_path, my_model)
        extracted_features[image_file] = features

with open('image_features.pkl', 'wb') as file:
    pickle.dump(extracted_features, file)





In [43]:
class TreeNode:
    def __init__(self, key):
        self.left = None
        self.right = None
        self.val = key


In [21]:
import pickle
import numpy as np

with open('image_features.pkl', 'rb') as f:
    features_dict = pickle.load(f)

normal_feat_dict = {}
for k, v in features_dict.items():
    if np.linalg.norm(v) > 0:
        normalized_vector = v / np.linalg.norm(v)
        normal_feat_dict[k] = normalized_vector
    else:
        normal_feat_dict[k] = v

with open('image_features_normalized.pkl', 'wb') as f:
    pickle.dump(normal_feat_dict, f)

In [52]:
pick = pd.read_pickle('image_features_normalized.pkl')
# pick

In [24]:
with open('image_features_normalized.pkl', 'rb') as file:
    normalized_features = pickle.load(file)

# **Q2**

In [25]:
def preprocess_text(text):
    """
    Preprocess text by lowercasing, tokenizing, removing stopwords, and stemming.
    """
    if not isinstance(text, str):
        return []
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return stemmed_tokens

result_folder = 'reviews'
os.makedirs(result_folder, exist_ok=True)

# Apply text processing
for idx, record in df.iterrows():
    text = process_text(str(record['Review Text']))
    file_location = os.path.join(result_folder, f"{record['Serial']}.txt")

    with open(file_location, 'w', encoding='utf-8') as file:
        file.write(text)


In [26]:
def calculate_tfidf_with_ids(document_data, doc_ids):
    """
    This function calculates the TF-IDF scores for given documents and associates them with document IDs.

    Parameters:
    - document_data: A list of preprocessed document texts.
    - doc_ids: A list of document IDs corresponding to the document texts.

    Returns:
    - A dictionary mapping document IDs to their TF-IDF score dictionaries.
    """
    # Calculate IDF for all tokens in all documents
    idf_counts = {}
    for document in document_data:
        seen_tokens = set()
        for token in document.split():
            if token not in seen_tokens:
                idf_counts[token] = idf_counts.get(token, 0) + 1
                seen_tokens.add(token)

    total_documents = len(document_data)
    idf = {token: math.log(total_documents / count) for token, count in idf_counts.items()}

    # Calculate TF-IDF for each document and associate it with its ID
    tfidf_documents = {}
    for document, doc_id in zip(document_data, doc_ids):
        tokens = document.split()
        token_counts = {token: tokens.count(token) for token in set(tokens)}
        total_tokens = len(tokens)
        tf = {token: count / total_tokens for token, count in token_counts.items()}

        tfidf = {token: tf_score * idf[token] for token, tf_score in tf.items()}
        tfidf_documents[doc_id] = tfidf

    return tfidf_documents

# Directory where processed text files are stored
data_folder = 'reviews'

# Process text files and calculate TF-IDF scores
processed_texts = []
ids = []
for filename in sorted(os.listdir(data_folder)):
    if filename.endswith('.txt'):
        doc_id = filename.split('.')[0]
        ids.append(doc_id)
        file_path = os.path.join(data_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            processed_texts.append(file.read())

# Calculate TF-IDF scores and associate them with document IDs
tfidf_results = calculate_tfidf_with_ids(processed_texts, ids)

# Save the TF-IDF scores to a pickle file
with open('tfidf_scores_with_ids.pkl', 'wb') as f:
    pickle.dump(tfidf_results, f)

print("TF-IDF scores calculated and saved successfully.")

In [44]:
class BinaryTree:
    def __init__(self):
        self.root = None

    def insert(self, key):
        if self.root is None:
            self.root = TreeNode(key)
        else:
            self._insert_recursive(self.root, key)

    def _insert_recursive(self, current_node, key):
        if key < current_node.val:
            if current_node.left is None:
                current_node.left = TreeNode(key)
            else:
                self._insert_recursive(current_node.left, key)
        else:
            if current_node.right is None:
                current_node.right = TreeNode(key)
            else:
                self._insert_recursive(current_node.right, key)

    def inorder_traversal(self, node):
        if node:
            self.inorder_traversal(node.left)
            print(node.val, end=' ')
            self.inorder_traversal(node.right)

    def preorder_traversal(self, node):
        if node:
            print(node.val, end=' ')
            self.preorder_traversal(node.left)
            self.preorder_traversal(node.right)

    def postorder_traversal(self, node):
        if node:
            self.postorder_traversal(node.left)
            self.postorder_traversal(node.right)
            print(node.val, end=' ')


In [ ]:
pick = pd.read_pickle('tfidf_scores_with_ids.pkl')
pick

# **Q3**

In [28]:
def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

## a)

In [54]:
# def top3_Images(id, img_features, top_k=3):
#     """Find the top_k most similar images to the input image."""
#     # Extract the feature vector for the input image
#     input_features = img_features[id]

#     # Compute cosine similarity with all other images
#     similarities = {}
#     for iden, features in img_features.items():
#         if iden != id:
#             similarity = cosine_similarity(input_features, features)
#             similarities[iden] = similarity

#     # Sort by similarity
#     sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)


#     # most_similar_images = sorted_similarities[:top_k]

#     # return most_similar_images

#     # Filter to ensure unique base IDs
#     all_ids = set(id.split('_')[0] for id in [id])
#     top_3_images = []
#     for image_id, similarity in sorted_similarities:
#         first = image_id.split('_')[0]
#         if first not in all_ids:
#             all_ids.add(first)
#             top_3_images.append((image_id, similarity))
#             if len(top_3_images) == top_k:
#                 break
#     return top_3_images


# # Load image features
# with open('image_features_normalized.pkl', 'rb') as f:
#     image_features = pickle.load(f)


# Image_Name = "5_1.jpg"
# top_similar_images = top3_Images(Image_Name, image_features)
# for id, similarity in top_similar_images:
#     print(f"Image Name: {id}, Similarity: {similarity}")

Image Name: 2091_1.jpg, Similarity: 0.769686758518219
Image Name: 1461_1.jpg, Similarity: 0.7078989148139954
Image Name: 3736_1.jpg, Similarity: 0.6948259472846985


## b)

In [55]:
# with open('tfidf_scores_with_ids.pkl', 'rb') as f:
#     pickle_tfidf = pickle.load(f)


# def cosine_similarity_sparse(v1, v2):
#     shared_keys = set(v1.keys()) & set(v2.keys())
#     dot_product = sum([v1[x] * v2[x] for x in shared_keys])

#     sum1 = sum([v1[x]**2 for x in v1.keys()])
#     sum2 = sum([v2[x]**2 for x in v2.keys()])
#     denominator = math.sqrt(sum1) * math.sqrt(sum2)

#     if not denominator:
#         return 0.0
#     else:
#         return float(dot_product) / denominator

# def top3_reviews(id, pickle_tfidf, top_k=3):
#     input_tfidf = pickle_tfidf[id]

#     similarities = {}
#     for review_id, tfidf in pickle_tfidf.items():
#         if review_id != id:
#             similarity = cosine_similarity_sparse(input_tfidf, tfidf)
#             similarities[review_id] = similarity

#     # Sort reviews by similarity
#     sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)

#     # Get the top_k similar review IDs
#     most_similar_reviews = sorted_similarities[:top_k]

#     return most_similar_reviews

# review_id = '5'
# most_similar_reviews = top3_reviews(review_id, pickle_tfidf)

# for id, similarity in most_similar_reviews:
#     print(f"Review ID: {id}, Similarity: {similarity}")

Review ID: 2498, Similarity: 0.15028397672226054
Review ID: 415, Similarity: 0.13626408419227173
Review ID: 1846, Similarity: 0.1228528391348387


In [31]:
review_corpus_sim, image_corpus_sim = {}, {}

for input_review_id in pickle_tfidf.keys():
    rws = top3_reviews(input_review_id, pickle_tfidf)
    review_corpus_sim[input_review_id] = rws

for input_id in image_features.keys():
    imgs = top3_Images(input_id, image_features)
    image_corpus_sim[input_id] = imgs


with open('review_only_retrieval.pkl', 'wb') as f:
    pickle.dump(review_corpus_sim, f)

with open('image_only_retrieval.pkl', 'wb') as f:
    pickle.dump(image_corpus_sim, f)


# **Q4**

In [57]:
# def avg_similarities(id, image_features, tfidf_scores):
#     # image similarities
#     img_similarities = {}
#     img_key = id + "_1.jpg"
#     primary_image = image_features[img_key]
#     for id1, features in image_features.items():
#         if id1 != img_key:
#             similar = cosine_similarity(primary_image, features)
#             img_similarities[id1.split('_')[0]] = similar

#     # review similarities
#     review_similarities = {}
#     primary_image_tfidf = tfidf_scores[id]
#     for id1, tfidf in tfidf_scores.items():
#         if id1 != id:
#             sim = cosine_similarity_sparse(primary_image_tfidf, tfidf)
#             review_similarities[id1] = sim

#     # average similarities
#     dict_avg_similarities = {}
#     for id1 in set(img_similarities.keys()).union(review_similarities.keys()):
#         avg_sim = (img_similarities.get(id1, 0) + review_similarities.get(id1, 0)) / 2
#         dict_avg_similarities[id1] = avg_sim
#     # show top threee
#     sorted_similarities = sorted(dict_avg_similarities.items(), key=lambda x: x[1], reverse=True)[:3]

#     return sorted_similarities

# Serial = '5'
# sorted_similarities = avg_similarities(Serial, image_features, pickle_tfidf)

# for id, sim in sorted_similarities:
#     print(f"ID: {id}, Average Similarity: {sim}")

# composite_similarities_all = {}
# for Serial in image_features.keys():
#     x = Serial.split('_')[0]
#     top3_similar = avg_similarities(x, image_features, pickle_tfidf)

#     composite_similarities_all[x] = top3_similar

# # Save the results
# with open('composite_retrieval.pkl', 'wb') as f:
#     pickle.dump(composite_similarities_all, f)

# # print(composite_similarities_all.keys())
# print(composite_similarities_all['4'])

ID: 1461, Average Similarity: 0.374909424174812
ID: 3736, Average Similarity: 0.35291199152910896
ID: 2498, Average Similarity: 0.3298056345822912
[('2883', 0.9999999999999999), ('3533', 0.41442079917752467), ('3580', 0.4134874523308613)]


# **Q5**

In [62]:
image_url = 'https://images-na.ssl-images-amazon.com/images/I/71raKRNMOPL._SY88.jpg'
input_review = '''I
have been using Fender locking tuners for about five years on various
strats and teles. Definitely helps with tuning stability and way faster to restring if
there is a break.'''
image_serial = (df[df['Image'].astype(str).str.contains(image_url)])['Serial'].unique().tolist()
review_serial = (df[df['Review Text'] == input_review])['Serial'].unique().tolist()
image_serial, review_serial

In [46]:
if __name__ == "__main__":
    # Create a binary tree instance
    bt = BinaryTree()

    # Insert nodes
    bt.insert(3)
    bt.insert(1)
    bt.insert(4)
    bt.insert(2)

    # print("In-order traversal:")
    # bt.inorder_traversal(bt.root)
    # print("\nPre-order traversal:")
    # bt.preorder_traversal(bt.root)
    # print("\nPost-order traversal:")
    # bt.postorder_traversal(bt.root)


In [47]:
import pandas as pd
import csv


image_retrieved_similarities = pd.read_pickle('image_only_retrieval.pkl')
with open('tfidf_scores_with_ids.pkl', 'rb') as f:
    pickle_tfidf = pickle.load(f)


input_code = url_id.split('_')[0]


with open('image_retrieval_results.csv', 'w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    # Write the header row
    csv_writer.writerow(['Input Image ID', 'Retrieved Image ID', 'Image URL(s)', 'Review', 'Cosine Similarity of Images', 'Cosine Similarity of Text', 'Composite Similarity Score'])

    for retrieved_id, similarity_score in image_retrieved_similarities[url_id]:
        x = retrieved_id.split('_')[0]
        image_urls = df[df['Serial'] == int(x)]['Image'].values
        reviews = df[df['Serial'] == int(x)]['Review Text'].values

        # Calculate text similarity
        text_similarity = cosine_similarity_sparse(pickle_tfidf[input_code], pickle_tfidf[str(x)])
        composite_similarity = (similarity_score + text_similarity) / 2

        # Prepare the data row
        image_urls_str = "; ".join(image_urls)
        reviews_str = " | ".join(reviews)

        # Write the data row to the CSV file
        csv_writer.writerow([url_id, retrieved_id, image_urls_str, reviews_str, f"{similarity_score:.4f}", f"{text_similarity:.4f}", f"{composite_similarity:.4f}"])

print("Results have been written to 'image_retrieval_results.csv'")


Results have been written to 'image_retrieval_results.csv'


In [51]:
import pandas as pd
import csv

# Load the pre-computed text similarities and image features
retrieved_text_similarities = pd.read_pickle('review_only_retrieval.pkl')
with open('image_features_normalized.pkl', 'rb') as f:
    image_features = pickle.load(f)

# Define the input text ID
input_text_id = '1205'

# Open a CSV file to write the results
with open('text_retrieval_results.csv', 'w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    # Write the header row
    csv_writer.writerow(['Input Text ID', 'Retrieved Text ID', 'Image URL(s)', 'Review', 'Cosine Similarity of Images', 'Cosine Similarity of Textual Content', 'Composite Similarity Score'])

    print(f"UNLEASHING TEXTUAL EXPLORATION [Input Text ID: {input_text_id}]:\n")

    for retrieved_id, similarity_score in retrieved_text_similarities[input_text_id]:
        # Retrieve image URLs and reviews for the retrieved ID
        image_urls = df[df['Serial'] == int(retrieved_id)]['Image'].values
        reviews = df[df['Serial'] == int(retrieved_id)]['Review Text'].values

        # Assuming a function `cosine_similarity` is already defined and accessible
        # Calculate the image similarity between the input and retrieved texts' associated images
        # Make sure to handle cases where the expected image feature vectors might not exist
        image_feature_1 = image_features.get(str(input_text_id) + "_1.jpg")
        image_feature_2 = image_features.get(str(retrieved_id) + "_1.jpg")
        if image_feature_1 is not None and image_feature_2 is not None:
            image_similarity = cosine_similarity(image_feature_1.reshape(1, -1), image_feature_2.reshape(1, -1))[0]
        else:
            image_similarity = 0  # Default to 0 if features are missing

        composite_similarity = (similarity_score + image_similarity) / 2

        # Prepare data for CSV
        image_urls_str = "; ".join(str(url) for url in image_urls)  # Concatenate all image URLs
        reviews_str = " | ".join(reviews)  # Concatenate all reviews

        # Write data to CSV
        csv_writer.writerow([input_text_id, retrieved_id, image_urls_str, reviews_str, f"{image_similarity:.4f}", f"{similarity_score:.4f}", f"{composite_similarity:.4f}"])

print("Text retrieval results have been written to 'text_retrieval_results.csv'")


Text retrieval results have been written to 'text_retrieval_results.csv'


As we can see image based retrival shows more accuracy than image